# Preprocessing Pandas

In [ ]:
import pandas as pd
sample_submission = pd.read_csv("understanding_cloud_organization/sample_submission.csv")
train = pd.read_csv("understanding_cloud_organization/train_320.csv")

In [ ]:
train['Image_name'] = train['Image_Label'].apply(lambda x: x.split('_')[0])
train['Label_name'] = train['Image_Label'].apply(lambda x: x.split('_')[1])
train.drop('Image_Label',axis=1,inplace=True)
train = train.pivot('Image_name','Label_name','EncodedPixels')

In [ ]:
train.head()

# Affichage Image

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

img=mpimg.imread('./understanding_cloud_organization/train_images_320/009e2f3.jpg')
img2=mpimg.imread('./understanding_cloud_organization/train_images_320_Black_Fish_Gravel_Sugar_Flower/009e2f3.jpg')

f, axarr = plt.subplots(1,2)
axarr[0].imshow(img)
axarr[1].imshow(img2)



# DATA LOADING

In [ ]:
from transformations import MoveAxis, Normalize01

In [3]:
from transformations import Compose, DenseTarget
from transformations import MoveAxis, Normalize01
from customdatasets import SegmentationDataSet
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pathlib

# root directory
root = pathlib.Path.cwd() / 'understanding_cloud_organization'
def get_filenames_of_path(path: pathlib.Path, ext: str = '*'):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

# input and target files
inputs = get_filenames_of_path(root / 'train_images_320')
targets = get_filenames_of_path(root / 'train_images_320_Black_Fish_Gravel_Sugar_Flower')
    

# training transformations and augmentations
transforms = Compose([
    DenseTarget(),
    Normalize01(),
    MoveAxis(),
])

# random seed
random_seed = 42

# split dataset into training set and validation set
train_size = 0.8  # 80:20 split

inputs_train, inputs_valid = train_test_split(
    inputs,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)


targets_train, targets_valid = train_test_split(
    targets,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

# dataset training
dataset_train = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms)

# dataset validation
dataset_valid = SegmentationDataSet(inputs=inputs_valid,
                                    targets=targets_valid,
                                    transform=transforms)

# dataloader training
dataloader_training = DataLoader(dataset=dataset_train,
                                 batch_size=1,
                                 shuffle=False)
# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_valid,
                                   batch_size=1,
                                   shuffle=False)



In [4]:
x, y = next(iter(dataloader_training))
print(f'x = shape: {x.shape}; type: {x.dtype}')
print(f'x = min: {x.min()}; max: {x.max()}')
print(f'y = shape: {y.shape}; class: {y.unique()}; type: {y.dtype}')
print(f'y = min: {y.min()}; max: {y.max()}')


(320, 480, 4)
[0. 1.]
x = shape: torch.Size([1, 3, 320, 480]); type: torch.float32
x = min: 0.0; max: 1.0
y = shape: torch.Size([1, 4, 320, 480]); class: tensor([0, 1]); type: torch.int64
y = min: 0; max: 1


## MODEL UNET

In [5]:
import torch
from unet import *
model = UNet(in_channels=3,
             out_channels=4,
             n_blocks=2,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2).cuda()

x = torch.randn(size=(1, 3, 320, 480), dtype=torch.float32).cuda()

with torch.no_grad():
    out = model(x)

print(f'Out: {out.shape}')

Out: torch.Size([1, 4, 320, 480])


In [6]:
from torchsummary import summary
summary = summary(model, (3, 320, 480))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 320, 480]             896
              ReLU-2         [-1, 32, 320, 480]               0
       BatchNorm2d-3         [-1, 32, 320, 480]              64
            Conv2d-4         [-1, 32, 320, 480]           9,248
              ReLU-5         [-1, 32, 320, 480]               0
       BatchNorm2d-6         [-1, 32, 320, 480]              64
         MaxPool2d-7         [-1, 32, 160, 240]               0
         DownBlock-8  [[-1, 32, 160, 240], [-1, 32, 320, 480]]               0
            Conv2d-9         [-1, 64, 160, 240]          18,496
             ReLU-10         [-1, 64, 160, 240]               0
      BatchNorm2d-11         [-1, 64, 160, 240]             128
           Conv2d-12         [-1, 64, 160, 240]          36,928
             ReLU-13         [-1, 64, 160, 240]               0
      BatchNorm2d-14    

In [ ]:
shape = 1920


def compute_max_depth(shape, max_depth=10, print_out=True):
    shapes = []
    shapes.append(shape)
    for level in range(1, max_depth):
        if shape % 2 ** level == 0 and shape / 2 ** level > 1:
            shapes.append(shape / 2 ** level)
            if print_out:
                print(f'Level {level}: {shape / 2 ** level}')
        else:
            if print_out:
                print(f'Max-level: {level - 1}')
            break

    return shapes


out = compute_max_depth(shape, print_out=True, max_depth=10)


In [ ]:
low = 10
high = 512
depth = 8


def compute_possible_shapes(low, high, depth):
    possible_shapes = {}
    for shape in range(low, high + 1):
        shapes = compute_max_depth(shape,
                                   max_depth=depth,
                                   print_out=False)
        if len(shapes) == depth:
            possible_shapes[shape] = shapes

    return possible_shapes
possible_shapes = compute_possible_shapes(low, high, depth)
possible_shapes


# TRAIN ET RUN

In [7]:
from train import Trainer

# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    torch.device('cpu')

# criterion
criterion = torch.nn.BCEWithLogitsLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# trainer
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  training_DataLoader=dataloader_training,
                  validation_DataLoader=dataloader_validation,
                  lr_scheduler=None,
                  epochs=4,
                  epoch=0,
                  notebook=True)

# start training
training_losses, validation_losses, lr_rates = trainer.run_trainer()


fqsfdsqfqsfsqfsfqsfq


(320, 480, 4)
[0. 1.]
tensor([[[[-6.3622e-01,  1.2339e-02, -3.4365e-01,  ...,  3.7109e-01,
           -9.9831e-01, -1.1909e+00],
          [ 7.8182e-01, -9.2661e-01,  2.4257e-01,  ...,  5.4175e-01,
           -3.6212e-01, -2.9613e-01],
          [-1.6041e+00, -4.9536e-01,  1.5249e+00,  ..., -3.0436e-01,
            1.5707e+00,  3.1424e-01],
          ...,
          [-5.7945e-01,  1.3291e+00, -7.2450e-01,  ..., -1.2481e+00,
            2.9970e-01, -5.9813e-02],
          [-6.7875e-01,  1.9287e+00,  1.5276e-01,  ..., -2.2289e+00,
           -1.0078e-01, -1.6999e-01],
          [ 4.3849e-01,  5.8851e-01,  3.8648e-01,  ...,  1.0841e+00,
           -5.9309e-01,  8.5957e-01]],

         [[ 8.3933e-02,  1.0792e+00,  4.7205e-01,  ...,  1.7602e-01,
           -9.3773e-01, -1.6699e-01],
          [-6.9396e-01, -8.7370e-01, -5.8831e-01,  ...,  2.6851e-01,
            4.3158e-02,  8.4035e-02],
          [ 5.5070e-01,  5.4432e-01,  1.4168e+00,  ...,  5.3776e-01,
            9.7221e-01, -8.3009e-02]

RuntimeError: result type Float can't be cast to the desired output type Long

# SAVE THE MODEL

In [ ]:
model_name =  'cloud.pt'
torch.save(model.state_dict(), pathlib.Path.cwd() / model_name)

# LEARNING RATE

In [ ]:
from lr_rate_finder import *
lrf = LearningRateFinder(model, criterion, optimizer, device)
lrf.fit(dataloader_training, steps=1000)

In [ ]:
lrf.plot()


In [ ]:
fig = plt.show(training_losses, validation_losses)
print(training_losses)
print(validation_losses)

In [ ]:
from visual import plot_training
fig = plot_training(training_losses, validation_losses, lr_rates, gaussian=True, sigma=1, figsize=(10, 4))